In [1]:
import sys
import gc
from datetime import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as tvmodels
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

sys.path.append('../script/')
import utils
import models
import model_parts as parts

fold = 0
DEVICE = "cuda"
EPOCHS = 3
params = {
    "num_layers": 3,
    "hidden_size": 16,
    "dropout": 0.3,
    "learning_rate": 1e-3
}

C:\Users\s3616\anaconda3\envs\moa\lib\site-packages\pytorch_lightning\utilities\distributed.py:36: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [2]:
df = pd.read_csv('../input/folds/train.csv')
df.columns

Index(['sig_id', 'g-0', 'g-1', 'g-2', 'g-3', 'g-4', 'g-5', 'g-6', 'g-7', 'g-8',
       ...
       'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor',
       'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b',
       'vitamin_d_receptor_agonist', 'wnt_inhibitor', 'kfold', 'nsc_labels'],
      dtype='object', length=1088)

In [3]:
print(f"[Fold No.{fold:>3}]\n")
train_df = df[df.kfold != fold].reset_index(drop=True)
valid_df = df[df.kfold == fold].reset_index(drop=True)

[Fold No.  0]



In [4]:
with open("../input/folds/targets", "r") as f:
    targets = f.read().split('\n')
with open("../input/folds/features", "r") as f:
    features = f.read().split('\n')

x_tr = train_df[features].to_numpy()
x_va = valid_df[features].to_numpy()

y_tr = train_df[targets].to_numpy()
y_va = valid_df[targets].to_numpy()

In [5]:
metrics_callback = utils.MetricsCallback()
tb_logger = pl_loggers.TensorBoardLogger("log/")
trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    weights_summary=None,  # "full" for enabling
    callbacks=[metrics_callback],
    progress_bar_refresh_rate=5,
    logger=tb_logger,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


---
## Prototype Model Here

In [6]:
# class ResNet18(nn.Module):
#     def __init__(self, num_features, num_targets, params):
#         super(ResNet18, self).__init__()
#         self.model = ResNet(num_features, BasicBlock, [2, 2, 2, 2], num_classes=num_targets)

#     def forward(self, x):
#         x = self.model(x)
#         return x

---

## Shape Test

In [7]:
base = models.BaseLine(x_tr.shape[1], y_tr.shape[1], params)
base(torch.zeros([16, x_tr.shape[1]])).shape

torch.Size([16, 207])

In [8]:
net = models.GIN(x_tr.shape[1], y_tr.shape[1], params)
net(torch.zeros([16, x_tr.shape[1]])).shape

TypeError: forward() missing 2 required positional arguments: 'edge_index' and 'batch'

In [ ]:
# model = utils.LitMoA(
#     hparams={},
#     model=net
# )
# dm = utils.MoADataModule(
#     hparams={"train_size": x_tr.shape[0], "batch_size": 1024},
#     data=np.vstack([x_tr, x_va]).copy(),
#     targets=np.vstack([y_tr, y_va]).copy(),
# )

# # # Free RAM space as much as possible before training
# # del df, train_df, valid_df, x_tr, x_va, y_tr, y_va
# # gc.collect()

In [ ]:
# trainer.fit(model, dm)